In [1]:
from SPARQLWrapper import SPARQLWrapper, SPARQLWrapper2, JSON, TURTLE, XML, RDFXML
import pprint
import csv
# from bs4 import BeautifulSoup
import matplotlib.pyplot as plt

from collections import Counter
from operator import itemgetter
import pandas as pd
from sqlalchemy import create_engine

In [39]:
query = """
PREFIX  egr:  <http://rdvocab.info/ElementsGr2/>
PREFIX  owl:  <http://www.w3.org/2002/07/owl#>
PREFIX  dbr:  <http://dbpedia.org/resource/>
PREFIX  xsd:  <http://www.w3.org/2001/XMLSchema#>
PREFIX  skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX  foaf: <http://xmlns.com/foaf/0.1/>

SELECT DISTINCT  ?s ?name ?sName ?uri ?year ?bio
WHERE
  {   { ?s  egr:biographicalInformation  ?bio ;
            egr:dateOfBirth       ?bd
        BIND(strbefore(strafter(str(?bd), "http://data.bnf.fr/date/"), "/") AS ?year)
        FILTER ( ?year > "1800" )
        FILTER ( ( ( regex(?bio, "juriste", "i") || regex(?bio, "Jurist", "i") ) || regex(?bio, "professeur de droit", "i") ) || regex(?bio, "docteur en droit", "i") )
        OPTIONAL
          { ?s  foaf:name  ?name }
        OPTIONAL
          { ?s  skos:prefLabel  ?sName }
        OPTIONAL
          { ?s  owl:sameAs  ?uri
            FILTER regex(?uri, "viaf.org", "i")
          }
      }
    UNION
      { ?s  egr:biographicalInformation  ?bio ;
            egr:dateOfBirth       ?bd
        BIND(strbefore(strafter(str(?bd), "http://data.bnf.fr/date/"), "/") AS ?year)
        FILTER ( ?year > "1800" )
        FILTER ( ( ( regex(?bio, "économiste", "i") || regex(?bio, "professeur d'économie", "i") ) || regex(?bio, "économie", "i") ) || regex(?bio, "docteur en économie", "i") )
        OPTIONAL
          { ?s  foaf:name  ?name }
        OPTIONAL
          { ?s  skos:prefLabel  ?sName }
        OPTIONAL
          { ?s  owl:sameAs  ?uri
            FILTER regex(?uri, "viaf.org", "i")
          }
      }
  }
ORDER BY DESC(?uri)
"""

In [3]:
sparql = SPARQLWrapper("https://data.bnf.fr/sparql") ##, returnFormat=RDFXML)  [LOCALHOST]

In [4]:
sparql.setQuery(query)
sparql.setReturnFormat(JSON)

In [5]:
try:
    rc_bnf = sparql.queryAndConvert()
except Exception as e:
    print(e)

In [6]:
# Number of rows in the result
len(rc_bnf['results']['bindings'])

7150

In [7]:
# Inspect the first three rows
i = 0
for l in rc_bnf['results']['bindings']:
    if i < 5:
        print(l)
        i += 1

{'s': {'type': 'uri', 'value': 'http://data.bnf.fr/ark:/12148/cb12981404c#about'}, 'name': {'type': 'literal', 'value': 'Léon Garnier'}, 'uri': {'type': 'uri', 'value': 'http://viaf.org/viaf/99996033'}, 'year': {'type': 'literal', 'value': '1836'}, 'bio': {'type': 'literal', 'value': "Juriste. - Administrateur et homme de lettres. - En poste à la Préfecture de la Seine. - Frère de l'explorateur Francis Garnier (1839-1873)"}}
{'s': {'type': 'uri', 'value': 'http://data.bnf.fr/ark:/12148/cb13484444m#about'}, 'name': {'type': 'literal', 'value': 'Gaston de Pawlowski'}, 'uri': {'type': 'uri', 'value': 'http://viaf.org/viaf/9999219'}, 'year': {'type': 'literal', 'value': '1874'}, 'bio': {'type': 'literal', 'value': 'Docteur en droit. - Critique littéraire et théâtral. - Rédacteur en chef de "Comoedia"'}}
{'s': {'type': 'uri', 'value': 'http://data.bnf.fr/ark:/12148/cb11919308t#about'}, 'name': {'type': 'literal', 'value': 'François Perroux'}, 'uri': {'type': 'uri', 'value': 'http://viaf.org

In [8]:
result_bnf = []
for l in rc_bnf['results']['bindings']:
        try: 
            name = l['name']['value']
        except Exception as e:
            name = ''
        try: 
            sName = l['sName']['value']
        except Exception as e:
            sName = ''
        try: 
            uri = l['uri']['value']
        except Exception as e:
            uri = ''
        try: 
            bio = l['bio']['value']
        except Exception as e:
            bio = ''
        try: 
            year = l['year']['value']
        except Exception as e:
            year = ''
        result_bnf.append([l['s']['value'], uri, name, sName, year, bio])        
            
        

In [9]:
print(len(result_bnf))
result_bnf[:2]

7150


[['http://data.bnf.fr/ark:/12148/cb12981404c#about',
  'http://viaf.org/viaf/99996033',
  'Léon Garnier',
  '',
  '1836',
  "Juriste. - Administrateur et homme de lettres. - En poste à la Préfecture de la Seine. - Frère de l'explorateur Francis Garnier (1839-1873)"],
 ['http://data.bnf.fr/ark:/12148/cb13484444m#about',
  'http://viaf.org/viaf/9999219',
  'Gaston de Pawlowski',
  '',
  '1874',
  'Docteur en droit. - Critique littéraire et théâtral. - Rédacteur en chef de "Comoedia"']]

In [10]:
query_2= """
PREFIX  dbo:  <http://dbpedia.org/ontology/>
PREFIX  dbp:  <http://dbpedia.org/property/>
PREFIX  owl:  <http://www.w3.org/2002/07/owl#>
PREFIX  dbr:  <http://dbpedia.org/resource/>
PREFIX  xsd:  <http://www.w3.org/2001/XMLSchema#>
PREFIX  foaf: <http://xmlns.com/foaf/0.1/>

SELECT DISTINCT  ?s ?uri ?name ?Birth_Date ?abstract
WHERE
      {   { ?s  a              dbo:Economist ;
                dbp:birthDate  ?Birth_Date
            FILTER ( xsd:date(?Birth_Date) > "1800-01-01"^^xsd:date )
            OPTIONAL
              { ?s  owl:sameAs  ?uri
                FILTER regex(?uri, "viaf", "i")
              }
            OPTIONAL
              { ?s  dbp:name  ?name } # I changed "foaf" to "dbp" because some "foaf" values are duplicated and one of the values is empty. 
               FILTER ( xsd:string(?name) != "Samuel Bowles" ) # I remove Samuel Bowles, because it added incorrect and dirty data cf. https://dbpedia.org/page/Samuel_Bowles_(economist). 
          }
        UNION
          { ?s  ?propriety     dbr:Economist ;
                dbp:birthDate  ?Birth_Date
            FILTER ( xsd:date(?Birth_Date) > "1800-01-01"^^xsd:date )
            OPTIONAL
              { ?s  owl:sameAs  ?uri
                FILTER regex(?uri, "viaf", "i")
                OPTIONAL
                  { ?s  dbp:name  ?name }
                FILTER ( xsd:string(?name) != "Samuel Bowles" )
              }
          }
        UNION
          { ?s  a              dbo:Professor ;
                dbp:birthDate  ?Birth_Date ;
                dbo:abstract   ?abstract
            FILTER ( ( ( ( ( regex(?abstract, "Economist", "i") || regex(?abstract, "économiste", "i") ) || regex(?abstract, "professeur d'économie", "i") ) || regex(?abstract, "economics professor", "i") ) || regex(?abstract, "docteur en économie", "i") ) || regex(?abstract, "PhD in Economics", "i") )
            OPTIONAL
              { ?s  dbp:name  ?name }
          FILTER ( xsd:string(?name) != "Samuel Bowles" )
            FILTER ( xsd:date(?Birth_Date) > "1800-01-01"^^xsd:date )
            OPTIONAL
              { ?s  owl:sameAs  ?uri
                FILTER regex(?uri, "viaf", "i")
              }
          }
        UNION
          { ?s  ?propriety     dbr:Jurist ;
                dbp:birthDate  ?Birth_Date
            FILTER ( xsd:date(?Birth_Date) > "1800-01-01"^^xsd:date )
            OPTIONAL
              { ?s  dbp:name  ?name
                FILTER ( xsd:string(?name) != "Marcus Tullius Cicero" ) # For some reason, Cicero appears into the data (certainly because his birth date begin with "-") so we remove him.
                FILTER ( xsd:string(?name) != "Samuel Bowles" )

              }
          }
        UNION
          { ?s  a              dbo:Lawyer ;
                dbp:birthDate  ?Birth_Date
            FILTER ( xsd:date(?Birth_Date) > "1800-01-01"^^xsd:date )
            OPTIONAL
              { ?s  dbp:name  ?name
                FILTER ( xsd:string(?name) != "Marcus Tullius Cicero" )
                FILTER ( xsd:string(?name) != "Samuel Bowles" )

              }
          }
        UNION
          { ?s  a              dbo:Professor ;
                dbp:birthDate  ?Birth_Date ;
                owl:sameAs     ?sameAs ;
                dbo:abstract   ?abstract
            FILTER ( ( ( ( ( contains(?abstract, "Jurist") || contains(?abstract, "Lawyer") ) || contains(?abstract, "lawyer") ) || contains(?abstract, "jurist") ) || contains(?abstract, "juriste") ) || contains(?abstract, "Juriste") )
            OPTIONAL
              { ?s  dbp:name  ?name
                FILTER ( xsd:string(?name) != "Marcus Tullius Cicero" )
                FILTER ( xsd:string(?name) != "Samuel Bowles" )
              }
            FILTER ( xsd:date(?Birth_Date) > "1800-01-01"^^xsd:date )
          }
      }
ORDER BY DESC(?uri)
"""

In [11]:
sparql = SPARQLWrapper("https://dbpedia.org/sparql")

In [12]:
sparql.setQuery(query_2)
sparql.setReturnFormat(JSON)

In [13]:
rc_db = sparql.queryAndConvert()

In [14]:
# Number of rows in the result
len(rc_db['results']['bindings'])

3560

In [15]:
# Inspect the first three rows
i = 0
for l in rc_db['results']['bindings']:
    if i <100:
        print(l)
        i += 1

{'s': {'type': 'uri', 'value': 'http://dbpedia.org/resource/Carlos_Carvalhas'}, 'uri': {'type': 'uri', 'value': 'http://viaf.org/viaf/99826658'}, 'name': {'type': 'literal', 'xml:lang': 'en', 'value': 'Carlos Carvalhas'}, 'Birth_Date': {'type': 'typed-literal', 'datatype': 'http://www.w3.org/2001/XMLSchema#date', 'value': '1941-11-09'}}
{'s': {'type': 'uri', 'value': 'http://dbpedia.org/resource/Mason_Gaffney'}, 'uri': {'type': 'uri', 'value': 'http://viaf.org/viaf/9960617'}, 'name': {'type': 'literal', 'xml:lang': 'en', 'value': 'Mason Gaffney'}, 'Birth_Date': {'type': 'typed-literal', 'datatype': 'http://www.w3.org/2001/XMLSchema#date', 'value': '1923-10-18'}}
{'s': {'type': 'uri', 'value': 'http://dbpedia.org/resource/Paulo_Portas'}, 'uri': {'type': 'uri', 'value': 'http://viaf.org/viaf/99455673'}, 'name': {'type': 'literal', 'xml:lang': 'en', 'value': 'Paulo Portas'}, 'Birth_Date': {'type': 'typed-literal', 'datatype': 'http://www.w3.org/2001/XMLSchema#date', 'value': '1962-09-12'}

In [16]:
result_dbpedia = []
for l in rc_db['results']['bindings']:
        try: 
            name = l['name']['value']
        except Exception as e:
            name = ''
        try: 
            birthDate = l['Birth_Date']['value']
        except Exception as e:
            birthDate = ''
        try: 
            uri = l['uri']['value']
        except Exception as e:
            uri = ''
        result_dbpedia.append([l['s']['value'], uri, name, birthDate])        
            
            


In [17]:
result_dbpedia[:3]

[['http://dbpedia.org/resource/Carlos_Carvalhas',
  'http://viaf.org/viaf/99826658',
  'Carlos Carvalhas',
  '1941-11-09'],
 ['http://dbpedia.org/resource/Mason_Gaffney',
  'http://viaf.org/viaf/9960617',
  'Mason Gaffney',
  '1923-10-18'],
 ['http://dbpedia.org/resource/Paulo_Portas',
  'http://viaf.org/viaf/99455673',
  'Paulo Portas',
  '1962-09-12']]

In [18]:
joined_data = []
for bnf in result_bnf:
    db_out = []
    for db in result_dbpedia:
        if bnf[1] == db[1] :
            db_out = db
    joined_data.append(bnf + db_out)

In [19]:
result_bnf[:2]

[['http://data.bnf.fr/ark:/12148/cb12981404c#about',
  'http://viaf.org/viaf/99996033',
  'Léon Garnier',
  '',
  '1836',
  "Juriste. - Administrateur et homme de lettres. - En poste à la Préfecture de la Seine. - Frère de l'explorateur Francis Garnier (1839-1873)"],
 ['http://data.bnf.fr/ark:/12148/cb13484444m#about',
  'http://viaf.org/viaf/9999219',
  'Gaston de Pawlowski',
  '',
  '1874',
  'Docteur en droit. - Critique littéraire et théâtral. - Rédacteur en chef de "Comoedia"']]

In [20]:
print(len(joined_data))

7150


In [21]:
joined_data[110]

['http://data.bnf.fr/ark:/12148/cb105592552#about',
 'http://viaf.org/viaf/9836899',
 'Jeanne Terrat-Branly',
 '',
 '1883',
 "Fille d'Edouard Branly (1844-1940), elle épouse, en 1906, le docteur en droit Georges Terrat"]

In [22]:
final_result = [i for i in joined_data if 'dbpedia' in i[-4]]
print(len(final_result))
pprint.pprint(final_result)

919
[['http://data.bnf.fr/ark:/12148/cb12432651s#about',
  'http://viaf.org/viaf/9939728',
  'Hermann Heinrich Gossen',
  '',
  '1810',
  'Économiste',
  'http://dbpedia.org/resource/Hermann_Heinrich_Gossen',
  'http://viaf.org/viaf/9939728',
  'Hermann Heinrich Gossen',
  '1810-09-07'],
 ['http://data.bnf.fr/ark:/12148/cb12279103q#about',
  'http://viaf.org/viaf/99257315',
  'Gottfried Haberler',
  '',
  '1900',
  'Économiste. - Professeur, Harvard university, Cambridge, Mass. (1936-1971). '
  '- Président, American economic association ; International economic '
  'association',
  'http://dbpedia.org/resource/Gottfried_Haberler',
  'http://viaf.org/viaf/99257315',
  'Gottfried Haberler',
  '1900-07-20'],
 ['http://data.bnf.fr/ark:/12148/cb108125478#about',
  'http://viaf.org/viaf/9838425',
  'Luigi Amoroso',
  '',
  '1886',
  "Mathématicien, professeur d'économie politique à l'Université de Rome "
  "(1926-1956). - Membre de l'Académie des Lincei (Rome)",
  'http://dbpedia.org/resour

In [23]:
final_result[:1]

[['http://data.bnf.fr/ark:/12148/cb12432651s#about',
  'http://viaf.org/viaf/9939728',
  'Hermann Heinrich Gossen',
  '',
  '1810',
  'Économiste',
  'http://dbpedia.org/resource/Hermann_Heinrich_Gossen',
  'http://viaf.org/viaf/9939728',
  'Hermann Heinrich Gossen',
  '1810-09-07']]

In [24]:
engine = create_engine('sqlite:///database.sqlite', echo=False)

In [25]:
df_bnf = pd.DataFrame([f for f in result_bnf if len(f[1]) > 0], columns=['uri_bnf', 'viaf', 'name_bnf', 'sName', 'year_bnf', 'bio_bnf'])
print(len(df_bnf))
df_bnf.fillna('')

df_bnf.head()

6432


,uri_bnf,viaf,name_bnf,sName,year_bnf,bio_bnf
0,http://data.bnf.fr/ark:/12148/cb12981404c#about,http://viaf.org/viaf/99996033,Léon Garnier,,1836,Juriste. - Administrateur et homme de lettres....
1,http://data.bnf.fr/ark:/12148/cb13484444m#about,http://viaf.org/viaf/9999219,Gaston de Pawlowski,,1874,Docteur en droit. - Critique littéraire et thé...
2,http://data.bnf.fr/ark:/12148/cb11919308t#about,http://viaf.org/viaf/99952938,François Perroux,,1903,Économiste. - Professeur d'économie politique ...
3,http://data.bnf.fr/ark:/12148/cb13379520q#about,http://viaf.org/viaf/9995247,Emmanuel Mathieu,,1852,"Docteur en droit (Paris, 1873)"
4,http://data.bnf.fr/ark:/12148/cb13338312g#about,http://viaf.org/viaf/9994322,Josiah Henry Benton,,1843,Juriste. - Bibliophile


In [26]:
df_bnf.to_sql('df_bnf', con=engine)

In [38]:
df_dbp = pd.DataFrame([p for p in result_dbpedia if len(p[1]) > 0], columns=['uri_dbp', 'viaf', 'name_dbp', 'date_dbp'])
print(len(df_dbp))
df_dbp.fillna('')
df_dbp

1862


,uri_dbp,viaf,name_dbp,date_dbp
0,http://dbpedia.org/resource/Carlos_Carvalhas,http://viaf.org/viaf/99826658,Carlos Carvalhas,1941-11-09
1,http://dbpedia.org/resource/Mason_Gaffney,http://viaf.org/viaf/9960617,Mason Gaffney,1923-10-18
2,http://dbpedia.org/resource/Paulo_Portas,http://viaf.org/viaf/99455673,Paulo Portas,1962-09-12
3,http://dbpedia.org/resource/Hermann_Heinrich_G...,http://viaf.org/viaf/9939728,Hermann Heinrich Gossen,1810-09-07
4,http://dbpedia.org/resource/Fernando_Teixeira_...,http://viaf.org/viaf/99275725,Fernando Teixeira dos Santos,1951-09-13
5,http://dbpedia.org/resource/Gottfried_Haberler,http://viaf.org/viaf/99257315,Gottfried Haberler,1900-07-20
6,http://dbpedia.org/resource/Michael_C._Burda,http://viaf.org/viaf/9922987,Michael C. Burda,1959-04-04
7,http://dbpedia.org/resource/Xavier_Vives,http://viaf.org/viaf/9920331,Xavier Vives,1955-01-23
8,http://dbpedia.org/resource/James_M._Poterba,http://viaf.org/viaf/9910825,James M. Poterba,1958-07-13
9,http://dbpedia.org/resource/David_Laibman,http://viaf.org/viaf/9910613,David Laibman,1942-12-25


In [28]:
df_dbp.to_sql('df_dbp', con=engine)

In [29]:
merged_df = pd.merge( df_dbp, df_bnf, on='viaf', how='inner')

In [37]:
print(len(merged_df))
merged_df

205


,uri_dbp,viaf,name_dbp,date_dbp,uri_bnf,name_bnf,sName,year_bnf,bio_bnf
0,http://dbpedia.org/resource/Hermann_Heinrich_G...,http://viaf.org/viaf/9939728,Hermann Heinrich Gossen,1810-09-07,http://data.bnf.fr/ark:/12148/cb12432651s#about,Hermann Heinrich Gossen,,1810,Économiste
1,http://dbpedia.org/resource/Gottfried_Haberler,http://viaf.org/viaf/99257315,Gottfried Haberler,1900-07-20,http://data.bnf.fr/ark:/12148/cb12279103q#about,Gottfried Haberler,,1900,"Économiste. - Professeur, Harvard university, ..."
2,http://dbpedia.org/resource/Luigi_Amoroso,http://viaf.org/viaf/9838425,Luigi Amoroso,1886-03-26,http://data.bnf.fr/ark:/12148/cb108125478#about,Luigi Amoroso,,1886,"Mathématicien, professeur d'économie politique..."
3,http://dbpedia.org/resource/Oskar_Morgenstern,http://viaf.org/viaf/98356107,Oskar Morgenstern,1902-01-24,http://data.bnf.fr/ark:/12148/cb119168550#about,Oskar Morgenstern,,1902,"Professeur d'économie, Universität Wien (1935-..."
4,http://dbpedia.org/resource/John_B._Condliffe,http://viaf.org/viaf/95173767,John Bell Condliffe,1891-12-23,http://data.bnf.fr/ark:/12148/cb11198287q#about,John Bell Condliffe,,1891,"Économiste, historien"
5,http://dbpedia.org/resource/Roy_Harrod,http://viaf.org/viaf/94499786,Sir Roy Harrod,1900-02-13,http://data.bnf.fr/ark:/12148/cb121817500#about,Henry Roy Forbes Harrod,,1900,Économiste
6,http://dbpedia.org/resource/Athanasios_Asimako...,http://viaf.org/viaf/93619789,Athanasios Asimakopulos,1930-05-28,http://data.bnf.fr/ark:/12148/cb12029492m#about,Athanasios Asimakopulos,,1930,"Économiste. - En poste, McGill University, Mon..."
7,http://dbpedia.org/resource/Johan_Beyen,http://viaf.org/viaf/93585571,Johan Beyen,1897-05-02,http://data.bnf.fr/ark:/12148/cb12424818s#about,Johan Willem Beyen,,1897,Économiste. - Ministre des Affaires étrangères...
8,http://dbpedia.org/resource/William_Vickrey,http://viaf.org/viaf/92276341,William Vickrey,1914-06-21,http://data.bnf.fr/ark:/12148/cb12475759c#about,William Vickrey,,1914,Économiste. - Professeur à la Columbia univers...
9,http://dbpedia.org/resource/Denis_Sargan,http://viaf.org/viaf/91851877,J. Denis Sargan,1924-08-23,http://data.bnf.fr/ark:/12148/cb12251561n#about,John Denis Sargan,,1924,A enseigné l'économie dans différentes univers...


In [36]:
merged_df[-40:]

,uri_dbp,viaf,name_dbp,date_dbp,uri_bnf,name_bnf,sName,year_bnf,bio_bnf
165,http://dbpedia.org/resource/Ragnar_Nurkse,http://viaf.org/viaf/14894400,Ragnar Nurkse,1907-10-05,http://data.bnf.fr/ark:/12148/cb12680064c#about,Ragnar Nurkse,,1907,Né en Estonie. - Etudes à Edimbourg et Vienne....
166,http://dbpedia.org/resource/Gérard_Debreu,http://viaf.org/viaf/14832766,Gérard Debreu,1921-07-04,http://data.bnf.fr/ark:/12148/cb122801806#about,Gerard Debreu,,1921,Professeur d'économie et de mathématiques à l'...
167,http://dbpedia.org/resource/Thomas_Nixon_Carver,http://viaf.org/viaf/12668196,Thomas Nixon Carver,1865-03-25,http://data.bnf.fr/ark:/12148/cb111973554#about,Thomas-Nixon Carver,,1865,Professeur d'économie politique à l'Université...
168,http://dbpedia.org/resource/Thorold_Rogers,http://viaf.org/viaf/12659238,Thorold Rogers,1823-03-23,http://data.bnf.fr/ark:/12148/cb12347168c#about,James Edwin Thorold Rogers,,1823,Economiste. - A été professeur d'économie poli...
169,http://dbpedia.org/resource/Karl_A._Fox,http://viaf.org/viaf/12402159,Karl A. Fox,1917-07-14,http://data.bnf.fr/ark:/12148/cb12446932h#about,Karl August Fox,,1917,Économiste. - Professeur à Iowa state universi...
170,http://dbpedia.org/resource/Tony_Atkinson,http://viaf.org/viaf/12353979,Tony Atkinson,1944-09-04,http://data.bnf.fr/ark:/12148/cb121949138#about,Anthony Barnes Atkinson,,1944,Économiste. - Professeur d'économie politique ...
171,http://dbpedia.org/resource/Émile_Louis_Victor...,http://viaf.org/viaf/12339853,Émile Louis Victor de Laveleye,1822-04-05,http://data.bnf.fr/ark:/12148/cb12117322n#about,Émile De Laveleye,,1822,Économiste. - Historien. - Professeur d'économ...
172,http://dbpedia.org/resource/Émile_Louis_Victor...,http://viaf.org/viaf/12339853,Emile de Laveleye,1822-04-05,http://data.bnf.fr/ark:/12148/cb12117322n#about,Émile De Laveleye,,1822,Économiste. - Historien. - Professeur d'économ...
173,http://dbpedia.org/resource/Augusto_Graziani,http://viaf.org/viaf/12320412,Augusto Graziani,1933-05-04,http://data.bnf.fr/ark:/12148/cb12019231n#about,Augusto Graziani,,1933,Économiste
174,http://dbpedia.org/resource/Bernard_Landry,http://viaf.org/viaf/119556759,Bernard Landry,1937-03-09,http://data.bnf.fr/ark:/12148/cb13499151g#about,Bernard Landry,,1937,"Diplômé en droit de l'Université de Montréal, ..."


In [32]:
df1 = pd.DataFrame({'user_id': ['id001', 'id002', 'id003', 'id004', 'id005', 'id006', 'id007'],
                    'first_name': ['Rivi', 'Wynnie', 'Kristos', 'Madalyn', 'Tobe', 'Regan', 'Kristin'],
                    'last_name': ['Valti', 'McMurty', 'Ivanets', 'Max', 'Riddich', 'Huyghe', 'Illis'],
                    'email': ['rvalti0@example.com', 'wmcmurty1@example.com', 'kivanets2@example.com',
                              'mmax3@example.com', 'triddich4@example.com', 'rhuyghe@example.com', 'killis4@example.com']
                    })

In [33]:
df2 = pd.DataFrame({'user_id': ['id001', 'id002', 'id003', 'id004', 'id005'],
                    'image_url': ['http://example.com/img/id001.png', 'http://example.com/img/id002.jpg',
                                  'http://example.com/img/id003.bmp', 'http://example.com/img/id004.jpg',
                                  'http://example.com/img/id005.png']
                    })

In [34]:
df3_merged = pd.merge(df1, df2)

In [35]:
df3_merged

,user_id,first_name,last_name,email,image_url
0,id001,Rivi,Valti,rvalti0@example.com,http://example.com/img/id001.png
1,id002,Wynnie,McMurty,wmcmurty1@example.com,http://example.com/img/id002.jpg
2,id003,Kristos,Ivanets,kivanets2@example.com,http://example.com/img/id003.bmp
3,id004,Madalyn,Max,mmax3@example.com,http://example.com/img/id004.jpg
4,id005,Tobe,Riddich,triddich4@example.com,http://example.com/img/id005.png
